# Read Classifier Data 

In [1]:
import os
from PIL import Image
import matplotlib.pyplot as plt

import torch
import torchvision
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision.transforms as transforms

#For converting the dataset to torchvision dataset format
class VowelConsonantDataset(Dataset):
    def __init__(self, file_path,train=True,transform=None):
        self.transform = transform
        self.file_path=file_path
        self.train=train
        self.file_names=[file for _,_,files in os.walk(self.file_path) for file in files]
        self.len = len(self.file_names)
        if self.train:
            self.classes_mapping=self.get_classes()
    def __len__(self):
        return len(self.file_names)
    
    def __getitem__(self, index):
        file_name=self.file_names[index]
        image_data=self.pil_loader(self.file_path+"/"+file_name)
        if self.transform:
            image_data = self.transform(image_data)
        if self.train:
            file_name_splitted=file_name.split("_")
            Y1 = self.classes_mapping[file_name_splitted[0]]
            Y2 = self.classes_mapping[file_name_splitted[1]]
            z1,z2=torch.zeros(10),torch.zeros(10)
            z1[Y1-10],z2[Y2]=1,1
            label=torch.stack([z1,z2])

            return image_data, label

        else:
            vow_test_tensor, con_test_tensor = torch.zeros(10,dtype=torch.int64), torch.zeros(10,dtype=torch.int64)
            numeric = file_name.split('.')[0]
            if len(numeric) < 4:
              numeric = '0'*(4-len(numeric))+numeric
            if numeric == '10000':
              numeric = '9999'
            vow_test_tensor[int(numeric[0])] = 1
            con_test_tensor[int(numeric[1])] = 1
            test_label = torch.stack([vow_test_tensor,con_test_tensor])
            return image_data, test_label
          
    def pil_loader(self,path):
        with open(path, 'rb') as f:
            img = Image.open(f)
            return img.convert('RGB')

      
    def get_classes(self):
        classes=[]
        for name in self.file_names:
            name_splitted=name.split("_")
            classes.extend([name_splitted[0],name_splitted[1]])
        classes=list(set(classes))
        classes_mapping={}
        for i,cl in enumerate(sorted(classes)):
            classes_mapping[cl]=i
        return classes_mapping
    

# Imports and Data download

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets

import torchvision.transforms as transforms

import numpy as np
import pandas as pd

train_on_gpu = torch.cuda.is_available()

In [3]:
import importlib
if importlib.util.find_spec('mlflow') is None:
  !pip install mlflow
import mlflow
import mlflow.pytorch

In [4]:
if importlib.util.find_spec('google.colab'):
  from google.colab import drive
  drive.mount('/content/drive')

In [5]:
!mkdir -p hin_classifier
!unzip -nq "/content/drive/My Drive/hin_classifier/train.zip" -d hin_classifier
!unzip -nq "/content/drive/My Drive/hin_classifier/test.zip" -d hin_classifier

/bin/sh: 1: unzip: Exec format error
/bin/sh: 1: unzip: Exec format error


# Utilities

In [6]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [7]:
def label_tensor(actual_labels):
    return torch.LongTensor([torch.max(labels, dim = -1)[1].item() for labels in actual_labels])

In [8]:
def cumulative_loss(predicted, actual, compute_loss=nn.CrossEntropyLoss()):
  actual_reshaped = actual.permute(1,0,2)
  vow_labels = label_tensor(actual_reshaped[0])
  con_labels = label_tensor(actual_reshaped[1])
  vow_labels = vow_labels.to(device)
  con_labels = con_labels.to(device)

  loss = compute_loss(predicted, vow_labels)
  loss += compute_loss(predicted, con_labels)
  return loss


In [9]:
def cumulative_correctness(predicted, actual, cur_value):
  actual_reshaped = actual.permute(1,0,2)
  vow_test = label_tensor(actual_reshaped[0].data).to(device)
  con_test = label_tensor(actual_reshaped[1].data).to(device)
  cur_value += predicted.eq(vow_test).sum().item()
  cur_value += predicted.eq(con_test).sum().item()
  return cur_value



# Train And Test Modules

In [10]:
class Params(object):
    def __init__(self, batch_size, epochs, seed, log_interval):
        self.batch_size = batch_size
        self.epochs = epochs
        self.seed = seed
        self.log_interval = log_interval

max_epochs = 16 if torch.cuda.is_available() else 2
args = Params(256, max_epochs, 0, 20)

In [11]:
def train(train_data_set,network, optimise, epoch, compute_loss):
    
  network.train()
  for batch_id, data in enumerate(train_data_set):
    inputs, labels = data
            
    optimise.zero_grad()
    inputs = inputs.to(device)
    outputs = network(inputs)
                  
    loss = cumulative_loss(outputs, labels)

    loss.backward()
    optimise.step()
      
    if batch_id % args.log_interval == 0:
      pos = epoch * len(train_data_set) + batch_id
      mlflow.log_metric('train_loss', loss.data.item()/len(inputs)*1000)
          
      print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.3f}'.format(
              epoch, batch_id * len(inputs), len(train_data_set.dataset),
              100. * batch_id / len(train_data_set), loss.data.item()))

In [12]:
def test(test_data_set, network, epoch, compute_loss, compute_correctness):
    
    network.eval()
    test_loss = 0
    correct = 0
    confusion_matrix = np.zeros([10, 10])
    
    with torch.no_grad():
        
        for inputs, labels in test_data_set:
            inputs = inputs.to(device)
            outputs = network(inputs)
            test_loss += compute_loss(outputs, labels).data.item()
            pred = outputs.data.max(1)[1]

            correct += compute_correctness(pred,labels,correct)
            
            for x, y in zip(pred.cpu().numpy(), labels.numpy()):
                confusion_matrix[x][y] += 1
            
        test_loss /= len(test_data_set.dataset)
        test_accuracy = 100.0 * correct / len(test_data_set.dataset)
        
        mlflow.log_metric('test_loss', test_loss*1000)
        mlflow.log_metric('test_accuracy', test_accuracy)
        
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.
              format(test_loss, correct, len(test_data_set.dataset), test_accuracy))
              
        if epoch == args.epochs:
            classes = np.arange(10)
            fig, ax = plt.subplots()
            im = ax.imshow(confusion_matrix, interpolation='nearest', cmap=plt.cm.Blues)
            ax.figure.colorbar(im, ax=ax)
            ax.set(xticks=np.arange(confusion_matrix.shape[1]),
                       yticks=np.arange(confusion_matrix.shape[0]),
                       xticklabels=classes, yticklabels=classes,
                       ylabel='True label',
                       xlabel='Predicted label',
                       title='Epoch %d' % epoch)
            thresh = confusion_matrix.max() / 2.
            for i in range(confusion_matrix.shape[0]):
                for j in range(confusion_matrix.shape[1]):
                    ax.text(j, i, int(confusion_matrix[i, j]),
                            ha="center", va="center",
                            color="white" if confusion_matrix[i, j] > thresh else "black")
              
            fig.tight_layout()
              
            image_path = 'images/%s.png' % (expt_id)
            plt.savefig(image_path)
            mlflow.log_artifact(image_path)


# Feed Forward Network

In [13]:
class FeedForwardNetwork(nn.Module):
    def __init__(self): 
        super(FeedForwardNetwork, self).__init__()
        self.classifier = nn.Sequential(
            nn.Linear(12288, 48),  # 64 x 64 x 3 = 12288
            nn.ReLU(),
            nn.Linear(48, 24),
            nn.ReLU(),
            nn.Linear(24, 10)
        )
             
    def forward(self, x):
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Run model with classifier data

In [14]:
transform = transforms.Compose([
    transforms.ToTensor()])

In [15]:
full_data = VowelConsonantDataset("hin_classifier/train",train=True,transform=transform)
train_size = int(0.9 * len(full_data))
test_size = len(full_data) - train_size

train_data, validation_data = random_split(full_data, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=args.batch_size, shuffle=True)
validation_loader = torch.utils.data.DataLoader(validation_data, batch_size=args.batch_size, shuffle=True)

In [16]:
test_data = VowelConsonantDataset("hin_classifier/test",train=False,transform=transform)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=args.batch_size,shuffle=False)

In [19]:
%%time
for epoch in range(args.epochs):
  ffn = FeedForwardNetwork().to(device)
  train(train_loader,ffn, optim.Adam(ffn.parameters()), epoch,cumulative_loss)
  test(test_loader,ffn, epoch,cumulative_loss, cumulative_correctness)

Train Epoch: 0 [0/9000 (0%)]	Loss: 4.607
Train Epoch: 0 [5120/9000 (56%)]	Loss: 4.613

Test set: Average loss: 0.0184, Accuracy: 7920791635818/10000 (79207916358%)

Train Epoch: 1 [0/9000 (0%)]	Loss: 4.640
Train Epoch: 1 [5120/9000 (56%)]	Loss: 4.602

Test set: Average loss: 0.0184, Accuracy: 14641755804652/10000 (146417558047%)

CPU times: user 35.1 s, sys: 12.2 s, total: 47.3 s
Wall time: 42.7 s


In [18]:
if importlib.util.find_spec('google.colab'):
  from google.colab import files
  files.download('mlruns')